<a href="https://colab.research.google.com/github/seonghunlee94/AI_study/blob/master/3%EC%A3%BC%EC%B0%A8_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

## 1. AG_News dataset 준비
### 1-1 Huggingface dataset의 fancyzhx/ag_news를 load합니다.

In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# [MY CODE] Huggingface dataset 로드
ds = load_dataset("fancyzhx/ag_news")

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### 1-2 `collate_fn` 함수에 다음 수정사항들을 반영하면 됩니다.
Truncation과 관련된 부분들을 지웁니다.

In [3]:
# [MY CODE] collate_fn
def collate_fn(batch):
    texts = [item['text'] for item in batch]
    labels = [item['label'] for item in batch]

    # 라벨 타입 확인
    # print(type(labels[0]))  # <class 'int'>이어야 함

    encoding = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,  # 필요에 따라 최대 길이 설정
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask'], torch.tensor(labels)


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [4]:
#model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
# DistilBERT 모델 로드
from transformers import AutoModelForSequenceClassification

# 4개의 클래스를 가지는 분류 모델 정의
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=4  # 클래스 수에 맞게 설정
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

model

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## 2. Classifier output, loss function, accuracy function 변경
### 2-1 TextClassifier의 출력 차원을 잘 조정하여 task를 풀 수 있도록 수정

In [5]:
from transformers import AutoModelForSequenceClassification
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self, model_name='distilbert-base-uncased', num_labels=4):
    super().__init__()

    # AutoModelForSequenceClassification으로 변경
    self.encoder = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels) # [MY CODE] 출력 차원 조정

  def forward(self, input_ids, attention_mask):
      # AutoModelForSequenceClassification은 이미 forward 메서드가 구현되어 있습니다.
      outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
      logits = outputs.logits  # logits는 분류 결과입니다.
      return logits


model = TextClassifier()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
for param in model.encoder.parameters():
  param.requires_grad = True

### 2-1  nn.CrossEntropyLoss 를 추가

In [6]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

lr = 1e-4
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()  # [MY CODE]  CrossEntropyLoss 사용

optimizer = Adam(model.parameters(), lr=lr)  # 전체 모델 학습
n_epochs = 10

for epoch in range(n_epochs):
    total_loss = 0.
    model.train()
    for data in train_loader:
        optimizer.zero_grad()  # model.zero_grad() 대신 optimizer.zero_grad()

        # 배치에서 input_ids, attention_mask, labels를 추출
        input_ids, attention_mask, labels = data
        input_ids, attention_mask, labels = input_ids.to('cuda'), attention_mask.to('cuda'), labels.to('cuda')

        # 모델에 입력하고 출력 얻기
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs  # 모델의 출력에서 logits을 가져옴

        # CrossEntropyLoss는 logits와 정수형 라벨을 받음
        loss = loss_fn(logits, labels)

        # 역전파 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 424.9995869472623
Epoch   1 | Train Loss: 257.87331570405513
Epoch   2 | Train Loss: 185.33991101477295
Epoch   3 | Train Loss: 130.10842373757623
Epoch   4 | Train Loss: 97.40959007560741
Epoch   5 | Train Loss: 75.24789609774598
Epoch   6 | Train Loss: 58.90112512240012
Epoch   7 | Train Loss: 49.71533935285697
Epoch   8 | Train Loss: 43.92870570415289
Epoch   9 | Train Loss: 40.979647502390435


### 2-2 정확도를 재는 accuracy 함수도 classification에 맞춰 수정하시면 됩니다.

## 3. 학습 결과 report

In [7]:
def accuracy(model, dataloader):
    cnt = 0
    acc = 0

    for input_ids, attention_mask, labels in dataloader:
        # 데이터를 GPU로 이동
        input_ids, attention_mask, labels = (
            input_ids.to('cuda'),
            attention_mask.to('cuda'),
            labels.to('cuda'),
        )

        # 모델 예측
        preds = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(preds, dim=-1)  # [MY CODE] classification에 맞춰 수정

        # 정확도 계산
        cnt += labels.shape[0]
        acc += (labels == preds).sum().item()

    return acc / cnt


with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.998 | Test acc: 0.933


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.